# Tabular Playground Series (January 2022)
## Import Data and Gather Insights

In [51]:
from typing import Dict, Tuple
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.neural_network import MLPRegressor

training_df = pd.read_csv("data/train.csv", index_col='row_id')
training_df['date'] = pd.to_datetime(training_df['date'], format='%Y-%m-%d')
display(training_df)

,date,country,store,product,num_sold
row_id,,,,,
0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329
1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520
2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146
3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572
4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911
...,...,...,...,...,...
26293,2018-12-31,Sweden,KaggleMart,Kaggle Hat,823
26294,2018-12-31,Sweden,KaggleMart,Kaggle Sticker,250
26295,2018-12-31,Sweden,KaggleRama,Kaggle Mug,1004


## Format Data

In [52]:
training_df['date'] = training_df['date'].dt.dayofyear / 365
countries = training_df.drop_duplicates(subset='country')['country']
stores = training_df.drop_duplicates(subset='store')['store']
products = training_df.drop_duplicates(subset='product')['product']

In [53]:
def hash_str(series: pd.Series) -> Tuple[Dict, Dict]:
    i = 0
    _map = {}
    reverse_map = {}
    step = 1 / len(series)
    for item in series:
        _map[item] = i
        reverse_map[i] = item
        i += step
    return _map, reverse_map

In [54]:
countries_map, countries_rmap = hash_str(countries)
stores_map, stores_rmap = hash_str(stores)
products_map, products_rmap = hash_str(products)
training_df['country'].replace(to_replace=countries_map, inplace=True)
training_df['store'].replace(to_replace=stores_map, inplace=True)
training_df['product'].replace(to_replace=products_map, inplace=True)
display(training_df)

,date,country,store,product,num_sold
row_id,,,,,
0,0.00274,0.000000,0.0,0.000000,329
1,0.00274,0.000000,0.0,0.333333,520
2,0.00274,0.000000,0.0,0.666667,146
3,0.00274,0.000000,0.5,0.000000,572
4,0.00274,0.000000,0.5,0.333333,911
...,...,...,...,...,...
26293,1.00000,0.666667,0.0,0.333333,823
26294,1.00000,0.666667,0.0,0.666667,250
26295,1.00000,0.666667,0.5,0.000000,1004


## Train MLP Regression Model

In [55]:
training_data = training_df[['date', 'country', 'store', 'product']].to_numpy()
target_values = training_df[['num_sold']].to_numpy().ravel()

In [56]:
mlp_regressor = MLPRegressor(hidden_layer_sizes=(4, 4),
                             activation='relu',
                             max_iter=400)
mlp_regressor = mlp_regressor.fit(training_data, target_values)

## Testing MLP Regressor Model

In [57]:
test_df = pd.read_csv("data/test.csv", index_col='row_id')
test_df['date'] = pd.to_datetime(test_df['date'], format='%Y-%m-%d')
display(test_df)

,date,country,store,product
row_id,,,,
26298,2019-01-01,Finland,KaggleMart,Kaggle Mug
26299,2019-01-01,Finland,KaggleMart,Kaggle Hat
26300,2019-01-01,Finland,KaggleMart,Kaggle Sticker
26301,2019-01-01,Finland,KaggleRama,Kaggle Mug
26302,2019-01-01,Finland,KaggleRama,Kaggle Hat
...,...,...,...,...
32863,2019-12-31,Sweden,KaggleMart,Kaggle Hat
32864,2019-12-31,Sweden,KaggleMart,Kaggle Sticker
32865,2019-12-31,Sweden,KaggleRama,Kaggle Mug


In [58]:
test_df['date'] = test_df['date'].dt.dayofyear / 365
test_df['country'].replace(to_replace=countries_map, inplace=True)
test_df['store'].replace(to_replace=stores_map, inplace=True)
test_df['product'].replace(to_replace=products_map, inplace=True)
display(test_df)

,date,country,store,product
row_id,,,,
26298,0.00274,0.000000,0.0,0.000000
26299,0.00274,0.000000,0.0,0.333333
26300,0.00274,0.000000,0.0,0.666667
26301,0.00274,0.000000,0.5,0.000000
26302,0.00274,0.000000,0.5,0.333333
...,...,...,...,...
32863,1.00000,0.666667,0.0,0.333333
32864,1.00000,0.666667,0.0,0.666667
32865,1.00000,0.666667,0.5,0.000000


In [59]:
test_data = test_df.to_numpy()
predictions = np.round(mlp_regressor.predict(test_data))
test_df['num_sold'] = np.int64(predictions)
test_df['date'] = test_df['date'] * 365
test_df['country'].replace(to_replace=countries_rmap, inplace=True)
test_df['store'].replace(to_replace=stores_rmap, inplace=True)
test_df['product'].replace(to_replace=products_rmap, inplace=True)
display(test_df)

,date,country,store,product,num_sold
row_id,,,,,
26298,1.0,Finland,KaggleMart,Kaggle Mug,379
26299,1.0,Finland,KaggleMart,Kaggle Hat,291
26300,1.0,Finland,KaggleMart,Kaggle Sticker,204
26301,1.0,Finland,KaggleRama,Kaggle Mug,589
26302,1.0,Finland,KaggleRama,Kaggle Hat,502
...,...,...,...,...,...
32863,365.0,Sweden,KaggleMart,Kaggle Hat,272
32864,365.0,Sweden,KaggleMart,Kaggle Sticker,185
32865,365.0,Sweden,KaggleRama,Kaggle Mug,570


In [60]:
test_df['num_sold'].to_csv('predictions.csv')